In [2]:
import os
import glob
import math
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd
import torch
from collections import defaultdict

import processing_io as pio
from torch_geometric.transforms import LineGraph

from torch_geometric.data import Data, Batch
import shapely.wkt as wkt
from tqdm import tqdm
import fiona
import os

import alphashape
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from shapely.geometry import Point
import random

# result_df_name = 'sim_output_1pm_capacity_reduction_10k_PRELIMINARY'
result_df_name = 'sim_output_1pm_capacity_reduction_10k_test'
result_path = '../../../../data/datasets_simulation_outputs/' + result_df_name
string_is_for_1pm = "pop_1pm"

base_dir_sample_sim_input = '../../../../data/' + string_is_for_1pm + '_simulations/' + string_is_for_1pm + '_policies_combinations_with_normal_dist/'
subdirs_pattern = os.path.join(base_dir_sample_sim_input, 'output_networks_*')
subdirs = list(set(glob.glob(subdirs_pattern)))
subdirs.sort()

gdf_basecase_output_links = gpd.read_file('results/' + string_is_for_1pm + '_basecase_average_output_links.geojson')
gdf_basecase_average_mode_stats = pd.read_csv('results/' + string_is_for_1pm + '_basecase_average_mode_stats.csv', delimiter=';')
districts = gpd.read_file("../../../../data/visualisation/districts_paris.geojson")

## Abstract

This is further than process_output_of_simulations_with_all_output_links_and_eqasim_info.ipynb, as it also includes more input information.

Note that there is more than one strategy to deal with the fact that there are more than one district per link. We implement the strategy of stacking the information of all districts together. 
An alternative strategy would be to use the mean of the information of the districts.

Process the districts manually, so that each link belongs to at most 3 districts.

## Process results

Process the outputs of the simulations for further usage by GNN.

In [3]:
# Read all network data into a dictionary of GeoDataFrames
def compute_result_dic():
    result_dic_output_links = {}
    result_dic_eqasim_trips = {}
    base_network_no_policies = gdf_basecase_output_links
    result_dic_output_links["base_network_no_policies"] = base_network_no_policies
    counter = 0
    for subdir in tqdm(subdirs, desc="Processing subdirs", unit="subdir"):
        counter += 1
        if counter > 5:
            break
        # print(f'Accessing folder: {subdir}')
        # print(len(os.listdir(subdir)))
        networks = [network for network in os.listdir(subdir) if not network.endswith(".DS_Store")]
        for network in networks:
            file_path = os.path.join(subdir, network)
            policy_key = pio.create_policy_key_1pm(network)
            df_output_links = pio.read_output_links(file_path)
            df_eqasim_trips = pio.read_eqasim_trips(file_path)
            if (df_output_links is not None and df_eqasim_trips is not None):
                df_output_links.drop(columns=['geometry'], inplace=True)
                gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_output_links, gdf_to_extend=df_output_links, column_to_extend='highway', new_column_name='highway')
                gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_output_links, gdf_to_extend=gdf_extended, column_to_extend='vol_car', new_column_name='vol_car_base_case')
                result_dic_output_links[policy_key] = gdf_extended
                mode_stats = pio.calculate_averaged_results(df_eqasim_trips)
                result_dic_eqasim_trips[policy_key] = mode_stats
    return result_dic_output_links, result_dic_eqasim_trips

result_dic_output_links, result_dic_eqasim_trips = compute_result_dic()
print(len(result_dic_eqasim_trips.keys()))
print(len(result_dic_output_links.keys()))

base_gdf = result_dic_output_links["base_network_no_policies"]
links_gdf_base = gpd.GeoDataFrame(base_gdf, geometry='geometry')
links_gdf_base.crs = "EPSG:2154"  # Assuming the original CRS is EPSG:2154
links_gdf_base.to_crs("EPSG:4326", inplace=True)
districts['district_centroid'] = districts['geometry'].centroid
links_gdf_with_districts = gpd.sjoin(links_gdf_base, districts, how='left', op='intersects')

# Group by edge and aggregate the district names
links_gdf_with_districts = links_gdf_with_districts.groupby('link').agg({
    'from_node': 'first',
    'to_node': 'first',
    'length': 'first',
    'freespeed': 'first',
    'capacity': 'first',
    'lanes': 'first',
    'modes': 'first',
    'vol_car': 'first',
    'highway': 'first',
    'geometry': 'first',
    'osm:way:oneway': 'first',
    'c_ar': lambda x: list(x.dropna()),
    'district_centroid': lambda x: list(x.dropna()),
    'perimetre': lambda x: list(x.dropna()),
    'surface': lambda x: list(x.dropna()),
}).reset_index()

gdf_now = gpd.GeoDataFrame(links_gdf_with_districts, geometry='geometry', crs=links_gdf_base.crs)
gdf_now = gdf_now.rename(columns={'c_ar': 'district', 'perimetre': 'district_perimeter', 'surface': 'district_surface'})

links_without_duplicates = pio.summarize_duplicate_edges(gdf_now)
print(f"Original number of edges: {len(gdf_now)}")
print(f"Number of edges after summarization: {len(links_without_duplicates)}")
remaining_duplicates = pio.find_duplicate_edges_in_gdf(links_without_duplicates)
print(f"Number of remaining duplicate edges: {len(remaining_duplicates)}")

Processing subdirs:   5%|▌         | 5/100 [01:06<21:10, 13.37s/subdir]


448
449


/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_7765/2992593321.py:37: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  districts['district_centroid'] = districts['geometry'].centroid
/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


Original number of edges: 31216
Number of edges after summarization: 25309
Number of remaining duplicate edges: 0


In [4]:
# PROCESS LINK GEOMETRIES
edge_start_point_tensor, stacked_edge_geometries_tensor, district_centroids_tensor_padded, edges_base, nodes = pio.get_link_geometries(links_without_duplicates, districts)

In [11]:
# Investigate the 'osm:way:oneway' column in links_gdf_base
# oneway_counts_base = links_gdf_base['osm:way:oneway'].value_counts(dropna=False)
# print("Counts in links_gdf_base['osm:way:oneway']:")
# print(oneway_counts_base)

# # Investigate the 'osm:way:oneway' column in links_gdf
# oneway_counts = links_gdf['osm:way:oneway'].value_counts(dropna=False)
# print("Counts in links_gdf['osm:way:oneway']:")
# print(oneway_counts)

# # Compare the counts
# comparison_df = pd.DataFrame({
#     'links_gdf_base': oneway_counts_base,
#     'links_gdf': oneway_counts
# }).fillna(0)
# print("Comparison of 'osm:way:oneway' counts between links_gdf_base and links_gdf:")
# print(comparison_df)

# # Check for roads where "from_node" is equal to "to_node"
# same_node_edges = links_without_duplicates[links_without_duplicates['from_node'] == links_without_duplicates['to_node']]
# print(f"Number of edges where 'from_node' is equal to 'to_node': {len(same_node_edges)}")

# # Check for edges which exist in both directions
# reverse_edges = links_without_duplicates.merge(links_without_duplicates, left_on=['from_node', 'to_node'], right_on=['to_node', 'from_node'], suffixes=('_1', '_2'))
# print(f"Number of edges which exist in both directions: {len(reverse_edges)}")

# Check for roads where "from_node" is equal to "to_node" in links_gdf_base
same_node_edges_base = links_gdf_base[links_gdf_base['from_node'] == links_gdf_base['to_node']]
print(f"Number of edges in links_gdf_base where 'from_node' is equal to 'to_node': {len(same_node_edges_base)}")

# Check for edges which exist in both directions in links_gdf_base
reverse_edges_base = links_gdf_base.merge(links_gdf_base, left_on=['from_node', 'to_node'], right_on=['to_node', 'from_node'], suffixes=('_1', '_2'))
print(f"Number of edges in links_gdf_base which exist in both directions: {len(reverse_edges_base)}")




# Check for roads where "from_node" is equal to "to_node" in links_gdf
same_node_edges = links_without_duplicates[links_without_duplicates['from_node'] == links_without_duplicates['to_node']]
print(f"Number of edges in links_gdf where 'from_node' is equal to 'to_node': {len(same_node_edges)}")

# Check for edges which exist in both directions in links_gdf
reverse_edges = links_without_duplicates.merge(links_without_duplicates, left_on=['from_node', 'to_node'], right_on=['to_node', 'from_node'], suffixes=('_1', '_2'))
print(f"Number of edges in links_gdf which exist in both directions: {len(reverse_edges)}")

print(f"Total number of links in links_gdf_base: {len(links_gdf_base)}")


Number of edges in links_gdf_base where 'from_node' is equal to 'to_node': 769
Number of edges in links_gdf_base which exist in both directions: 12595
Number of edges in links_gdf where 'from_node' is equal to 'to_node': 766
Number of edges in links_gdf which exist in both directions: 766
Total number of links in links_gdf_base: 31216


In [18]:
reverse_edges_base.head()

,link_1,from_node_1,to_node_1,length_1,freespeed_1,capacity_1,lanes_1,modes_1,vol_car_1,osm:relation:route_master_1,...,osm:way:id_2,osm:way:access_2,osm:way:oneway_2,highway_2,osm:relation:route_2,osm:way:railway_2,osm:way:name_2,storageCapacityUsedInQsim_2,osm:way:tunnel_2,geometry_2
0,100316,5904976363,24983651,14.860209,8.333333,480.0,1.0,"bus,car,car_passenger,pt",9.607843,NaN,...,4216831.0,NaN,NaN,tertiary,bus,NaN,Carrefour de l'Odéon,NaN,NaN,"LINESTRING (2.33872 48.85229, 2.33874 48.85242)"
1,100317,24983651,5904976363,14.860209,8.333333,960.0,2.0,"bus,car,car_passenger,pt",2.490196,NaN,...,4216831.0,NaN,NaN,tertiary,bus,NaN,Carrefour de l'Odéon,NaN,NaN,"LINESTRING (2.33874 48.85242, 2.33872 48.85229)"
2,100586,125496888,116126870,303.962262,8.333333,480.0,1.0,"bus,car,car_passenger,pt",0.843137,NaN,...,41251681.0,NaN,NaN,residential,bus,NaN,Rue de Cronstadt,NaN,NaN,"LINESTRING (2.30279 48.83540, 2.30091 48.83296)"
3,100587,116126870,125496888,303.962262,8.333333,480.0,1.0,"bus,car,car_passenger,pt",3.627451,NaN,...,41251681.0,NaN,NaN,residential,bus,NaN,Rue de Cronstadt,NaN,NaN,"LINESTRING (2.30091 48.83296, 2.30279 48.83540)"
4,100966,268234194,1343567797,21.485633,8.333333,480.0,1.0,"bus,car,car_passenger,pt",1.823529,NaN,...,24606807.0,NaN,NaN,tertiary,bus,NaN,Avenue de la Porte de Montmartre,NaN,NaN,"LINESTRING (2.33629 48.89915, 2.33640 48.89898)"


In [17]:
# Print a few pairs of edges which exist in both directions in links_gdf_base
if not reverse_edges_base.empty:
    print("Sample pairs of edges in links_gdf_base which exist in both directions:")
    for i in range(min(5, len(reverse_edges_base))):
        sample_pair = reverse_edges_base.iloc[i]
        print(f"\nPair {i+1}:")
        print(f"Edge 1: from_node = {sample_pair['from_node_1']}, to_node = {sample_pair['to_node_1']}")
        print(f"Edge 2: from_node = {sample_pair['from_node_2']}, to_node = {sample_pair['to_node_2']}")
        
        # Compare some attributes
        attrs_to_compare = ['length', 'freespeed', 'capacity', 'lanes', 'modes', 'highway']
        for attr in attrs_to_compare:
            if attr in sample_pair.index:
                print(f"{attr}: Edge 1 = {sample_pair[f'{attr}_1']}, Edge 2 = {sample_pair[f'{attr}_2']}")
else:
    print("No edges in links_gdf_base exist in both directions.")

Sample pairs of edges in links_gdf_base which exist in both directions:

Pair 1:
Edge 1: from_node = 5904976363, to_node = 24983651
Edge 2: from_node = 24983651, to_node = 5904976363

Pair 2:
Edge 1: from_node = 24983651, to_node = 5904976363
Edge 2: from_node = 5904976363, to_node = 24983651

Pair 3:
Edge 1: from_node = 125496888, to_node = 116126870
Edge 2: from_node = 116126870, to_node = 125496888

Pair 4:
Edge 1: from_node = 116126870, to_node = 125496888
Edge 2: from_node = 125496888, to_node = 116126870

Pair 5:
Edge 1: from_node = 268234194, to_node = 1343567797
Edge 2: from_node = 1343567797, to_node = 268234194


In [19]:
if not reverse_edges_base.empty:
    print("Analyzing pairs of edges in links_gdf_base which exist in both directions:")
    attrs_to_compare = ['length', 'freespeed', 'capacity', 'lanes', 'modes', 'highway']
    differences_found = False

    for i in range(len(reverse_edges_base)):
        sample_pair = reverse_edges_base.iloc[i]
        differences = []

        for attr in attrs_to_compare:
            if attr in sample_pair.index:
                if sample_pair[f'{attr}_1'] != sample_pair[f'{attr}_2']:
                    differences.append(f"{attr}: Edge 1 = {sample_pair[f'{attr}_1']}, Edge 2 = {sample_pair[f'{attr}_2']}")

        if differences:
            differences_found = True
            print(f"\nPair {i+1}:")
            print(f"Edge 1: from_node = {sample_pair['from_node_1']}, to_node = {sample_pair['to_node_1']}")
            print(f"Edge 2: from_node = {sample_pair['from_node_2']}, to_node = {sample_pair['to_node_2']}")
            print("Differences found:")
            for diff in differences:
                print(f"  {diff}")

    if not differences_found:
        print("All pairs of bidirectional edges have equivalent attributes.")
else:
    print("No edges in links_gdf_base exist in both directions.")

# Print summary statistics
print("\nSummary:")
print(f"Total number of bidirectional edge pairs: {len(reverse_edges_base)}")
print(f"Number of pairs with differences: {differences_found}")

Analyzing pairs of edges in links_gdf_base which exist in both directions:
All pairs of bidirectional edges have equivalent attributes.

Summary:
Total number of bidirectional edge pairs: 12595
Number of pairs with differences: False


In [20]:
import numpy as np

if not reverse_edges_base.empty:
    print("Analyzing pairs of edges in links_gdf_base which exist in both directions:")
    attrs_to_compare = ['length', 'freespeed', 'capacity', 'lanes', 'modes', 'highway']
    differences_found = 0
    total_pairs = len(reverse_edges_base)

    for i in range(total_pairs):
        sample_pair = reverse_edges_base.iloc[i]
        differences = []

        for attr in attrs_to_compare:
            if attr in sample_pair.index:
                val1 = sample_pair[f'{attr}_1']
                val2 = sample_pair[f'{attr}_2']
                
                if isinstance(val1, (int, float)) and isinstance(val2, (int, float)):
                    if not np.isclose(val1, val2, rtol=1e-5, atol=1e-8):
                        differences.append(f"{attr}: Edge 1 = {val1}, Edge 2 = {val2}")
                elif val1 != val2:
                    differences.append(f"{attr}: Edge 1 = {val1}, Edge 2 = {val2}")

        if differences:
            differences_found += 1
            if differences_found <= 5:  # Print details for first 5 differing pairs
                print(f"\nPair {i+1}:")
                print(f"Edge 1: from_node = {sample_pair['from_node_1']}, to_node = {sample_pair['to_node_1']}")
                print(f"Edge 2: from_node = {sample_pair['from_node_2']}, to_node = {sample_pair['to_node_2']}")
                print("Differences found:")
                for diff in differences:
                    print(f"  {diff}")
        
        if i < 5:  # Print first 5 pairs regardless of differences
            print(f"\nSample Pair {i+1}:")
            for attr in attrs_to_compare:
                if attr in sample_pair.index:
                    print(f"  {attr}: Edge 1 = {sample_pair[f'{attr}_1']}, Edge 2 = {sample_pair[f'{attr}_2']}")

    print("\nSummary:")
    print(f"Total number of bidirectional edge pairs: {total_pairs}")
    print(f"Number of pairs with differences: {differences_found}")
    if differences_found == 0:
        print("All pairs of bidirectional edges have equivalent attributes.")
else:
    print("No edges in links_gdf_base exist in both directions.")

Analyzing pairs of edges in links_gdf_base which exist in both directions:

Sample Pair 1:

Sample Pair 2:

Sample Pair 3:

Sample Pair 4:

Sample Pair 5:

Summary:
Total number of bidirectional edge pairs: 12595
Number of pairs with differences: 0
All pairs of bidirectional edges have equivalent attributes.


In [14]:
# Print a pair of edges which exist in both directions in links_gdf_base
# if not reverse_edges_base.empty:
#     sample_pair_base = reverse_edges_base.iloc[0]
#     print(f"Sample pair of edges in links_gdf_base which exist in both directions:")
#     print(f"Edge 1: from_node = {sample_pair_base['from_node_1']}, to_node = {sample_pair_base['to_node_1']}")
#     print(f"Edge 2: from_node = {sample_pair_base['from_node_2']}, to_node = {sample_pair_base['to_node_2']}")
# else:
#     print("No edges in links_gdf_base exist in both directions.")
    
# # Print the whole row of edges which exist in both directions in links_gdf_base
# if not reverse_edges_base.empty:
#     sample_pair_base = reverse_edges_base.iloc[0]
#     print(f"Sample pair of edges in links_gdf_base which exist in both directions:")
#     print(sample_pair_base)
# else:
#     print("No edges in links_gdf_base exist in both directions.")


# Compare the attributes of pairs of edges which exist in both directions in links_gdf_base
# if not reverse_edges_base.empty:
#     for idx, row in reverse_edges_base.iterrows():
#         edge_1_attrs = links_gdf_base[(links_gdf_base['from_node'] == row['from_node_1']) & (links_gdf_base['to_node'] == row['to_node_1'])].iloc[0]
#         edge_2_attrs = links_gdf_base[(links_gdf_base['from_node'] == row['from_node_2']) & (links_gdf_base['to_node'] == row['to_node_2'])].iloc[0]
        
#         print(f"Comparing attributes for edge pair {idx + 1}:")
#         for attr in edge_1_attrs.index:
#             if edge_1_attrs[attr] != edge_2_attrs[attr]:
#                 print(f"Attribute '{attr}' differs: Edge 1 = {edge_1_attrs[attr]}, Edge 2 = {edge_2_attrs[attr]}")
# else:
#     print("No edges in links_gdf_base exist in both directions.")




Comparing attributes for edge pair 1:
Attribute 'link' differs: Edge 1 = 100316, Edge 2 = 100317
Attribute 'from_node' differs: Edge 1 = 5904976363, Edge 2 = 24983651
Attribute 'to_node' differs: Edge 1 = 24983651, Edge 2 = 5904976363
Attribute 'capacity' differs: Edge 1 = 480.0, Edge 2 = 960.0
Attribute 'lanes' differs: Edge 1 = 1.0, Edge 2 = 2.0
Attribute 'vol_car' differs: Edge 1 = 9.607843137254902, Edge 2 = 2.4901960784313726
Attribute 'osm:relation:route_master' differs: Edge 1 = nan, Edge 2 = nan
Attribute 'osm:way:vehicle' differs: Edge 1 = nan, Edge 2 = nan
Attribute 'osm:way:traffic_calming' differs: Edge 1 = nan, Edge 2 = nan
Attribute 'osm:way:junction' differs: Edge 1 = nan, Edge 2 = nan
Attribute 'osm:way:motorcycle' differs: Edge 1 = nan, Edge 2 = nan
Attribute 'osm:way:psv' differs: Edge 1 = nan, Edge 2 = nan
Attribute 'osm:way:service' differs: Edge 1 = nan, Edge 2 = nan
Attribute 'osm:way:access' differs: Edge 1 = nan, Edge 2 = nan
Attribute 'osm:way:oneway' differs: 

KeyboardInterrupt: 

# Analyze results and save to file

In [6]:
# pio.analyze_geodataframes(result_dic=result_dic, consider_only_highway_edges=True)

In [7]:
# pio.analyze_geodataframes(result_dic=result_dic, consider_only_highway_edges=False)

In [8]:
def process_result_dic(result_dic, result_dic_mode_stats, save_path=None, batch_size=500, gdf_input=None):
    os.makedirs(save_path, exist_ok=True)
    datalist = []
    linegraph_transformation = LineGraph()
    
    vol_base_case = links_without_duplicates['vol_car'].values
    capacity_base_case = np.where(links_without_duplicates['modes'].str.contains('car'), links_without_duplicates['capacity'], 0)
    length = links_without_duplicates['length'].values
    freespeed_base_case = links_without_duplicates['freespeed'].values
    allowed_modes = pio.encode_modes(links_without_duplicates)
    edge_index = torch.tensor(edges_base, dtype=torch.long).t().contiguous()
    x = torch.zeros((len(nodes), 1), dtype=torch.float)
    data = Data(edge_index=edge_index, x=x)
    
    batch_counter = 0
    for key, df in tqdm(result_dic.items(), desc="Processing result_dic", unit="dataframe"):   
        if isinstance(df, pd.DataFrame) and key != "base_network_no_policies":
            gdf = pio.prepare_gdf(df, gdf_input)
            len_edges = len(gdf)
            
            capacities_new, capacity_reduction, highway, freespeed =  pio.get_basic_edge_attributes(capacity_base_case, gdf)
            district_info, combined_tensor = pio.compute_combined_tensor(vol_base_case, capacity_base_case, length, freespeed_base_case, allowed_modes, gdf, capacities_new, capacity_reduction, highway, freespeed)
            
            linegraph_data = linegraph_transformation(data)
            linegraph_data.x = combined_tensor
            linegraph_data.num_nodes = combined_tensor.shape[0]
        
            # add edge attributes: 1 if edge to district, 0 if edge to edge
            edge_to_district_index, edge_to_district_attr = pio.compute_edge_attributes(district_info, linegraph_data, len_edges, gdf)
            if linegraph_data.edge_attr is None:
                linegraph_data.edge_attr = torch.zeros((linegraph_data.edge_index.shape[1] - edge_to_district_index.shape[1], 1), dtype=torch.long)
            linegraph_data.edge_attr = torch.cat([linegraph_data.edge_attr, edge_to_district_attr], dim=0)

            # add node attributes: 1 if district, 0 if edge
            node_type_feature = pio.compute_node_attributes(district_info, len_edges)
            linegraph_data.x = torch.cat([linegraph_data.x, node_type_feature], dim=1)
            linegraph_data.pos = torch.cat([stacked_edge_geometries_tensor, district_centroids_tensor_padded], dim=0)
            linegraph_data.y = pio.compute_target_tensor(vol_base_case, gdf, district_info)
                        
            df_mode_stats = result_dic_mode_stats.get(key)
            if df_mode_stats is not None:
                numeric_cols = df_mode_stats.select_dtypes(include=[np.number]).columns
                mode_stats_numeric = df_mode_stats[numeric_cols].astype(float)
                mode_stats_tensor = torch.tensor(mode_stats_numeric.values, dtype=torch.float)
                linegraph_data.mode_stats = mode_stats_tensor
            
            if linegraph_data.validate(raise_on_error=True):
                datalist.append(linegraph_data)
                batch_counter += 1

                # Save intermediate result every batch_size data points
                if batch_counter % batch_size == 0:
                    batch_index = batch_counter // batch_size
                    torch.save(datalist, os.path.join(save_path, f'datalist_batch_{batch_index}.pt'))
                    datalist = []  # Reset datalist for the next batch
            else:
                print("Invalid line graph data")
    
    # Save any remaining data points
    if datalist:
        batch_index = (batch_counter // batch_size) + 1
        torch.save(datalist, os.path.join(save_path, f'datalist_batch_{batch_index}.pt'))
    return

# Call the function
process_result_dic(result_dic=result_dic_output_links, result_dic_mode_stats=result_dic_eqasim_trips, save_path=result_path, batch_size=200, gdf_input=links_without_duplicates)

6


Processing result_dic:   1%|          | 3/449 [00:06<16:35,  2.23s/dataframe]


KeyboardInterrupt: 